# LangChain 实战：基于场景化的聊天机器人（房产，英语培训，手机，及汽车）


## 使用 Document Transformers 模块来处理原始数据的预处理，包括 房产，英语培训，手机，及汽车

In [1]:
with open("real_estate_sales_data.txt") as f:
    real_estate_sales = f.read()

In [2]:
with open("english_training_sales_data.txt") as f1:
    english_training_sales = f1.read()

In [3]:
with open("iphone_sales_data.txt") as f2:
    iphone_sales = f2.read()

In [55]:
with open("car_sales_data.txt") as f3:
    car_sales = f3.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [4]:
from langchain.text_splitter import CharacterTextSplitter

In [7]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 200,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [8]:
docs_real = text_splitter.create_documents([real_estate_sales])

docs_english = text_splitter.create_documents([english_training_sales])

docs_iphone = text_splitter.create_documents([iphone_sales])

In [56]:
docs_car = text_splitter.create_documents([car_sales])

In [11]:
docs_real[0]

Document(metadata={}, page_content='[客户问题] 这个小区交通便利吗？\n[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。\n\n\\d+\\.  \n[客户问题] 我担心楼下太吵。\n[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。')

In [12]:
docs_english[0]

Document(metadata={}, page_content='[客户问题] 这个培训课程的费用是多少？\n[销售回答] 课程费用因课程类型和时长而异，不过我们可以为您提供详细的价格清单和不同付款选项，您方便告诉我您感兴趣的课程吗？\n\n\\d+\\.\n[客户问题] 这个培训课程的时长是多久？\n[销售回答] 课程的时长因不同课程而异，通常从几周到几个月不等。我们可以根据您的需求提供最适合您的课程时长，您有特定的时间要求吗？')

In [13]:
docs_iphone[0]

Document(metadata={}, page_content='[客户问题] 这款手机电池续航怎么样？\n[销售回答] 这款iPhone使用了最先进的电池技术，可以保证您在正常使用下整天都不用担心电量问题。\n\n\\d+\\.\n[客户问题] 这款手机的相机效果如何？\n[销售回答] iPhone的相机一直都是业界标杆，它配备了先进的摄像头系统，无论是白天还是夜晚，都可以拍出清晰美丽的照片。')

In [57]:
docs_car[0]

Document(metadata={}, page_content='[客户问题] 车辆的玻璃水需要加特定品牌的吗？  \n[销售回答] 不一定非要加特定品牌的玻璃水，但建议选择质量可靠、符合车辆使用要求的产品。  \n\n\\d+\\.  \n[客户问题] 新车磨合期需要注意什么？  \n[销售回答] 磨合期内建议避免急加速和急刹车，保持发动机转速在合理范围，首次保养后即可正常驾驶。')

In [14]:
len(docs_real)

25

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [15]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db_real = FAISS.from_documents(docs_real, OpenAIEmbeddings())

/var/folders/hs/8p529r5x2811ltb4jwc49lgr0000gn/T/ipykernel_8364/3634384091.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db_real = FAISS.from_documents(docs_real, OpenAIEmbeddings())


In [16]:
db_english = FAISS.from_documents(docs_english, OpenAIEmbeddings())

In [17]:
db_iphonoe = FAISS.from_documents(docs_iphone, OpenAIEmbeddings())

In [58]:
db_car = FAISS.from_documents(docs_car, OpenAIEmbeddings())

In [20]:
query = "小区吵不吵"

In [21]:
answer_list = db_real.similarity_search(query)

In [22]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

\d+\.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

[客户问题] 我喜欢安静，这里噪音大吗？
[销售回答] 我们特意进行了隔音设计，并且小区内部也有绿化带，整体非常安静。

\d+\.  
[客户问题] 我看这个小区还在建设中。
[销售回答] 是的，目前正在建设中，但已经接近尾声，很快就会完工。

\d+\.  
[客户问题] 有哪些休闲设施？
[销售回答] 我们有游泳池、健身房、图书馆以及儿童游乐场等多种休闲设施。

[客户问题] 我听说这个区域交通常常堵塞。
[销售回答] 我们小区的位置实际上是绕开了主要交通瓶颈区，所以相对来说会流畅很多。

\d+\.  
[客户问题] 我很关注环境保护，你们有做到吗？
[销售回答] 非常好的问题，我们的建筑采用环保材料，并且在小区规划中也有绿化区域和垃圾分类系统。

[客户问题] 我看周围没有学校。
[销售回答] 其实附近就有几所知名的学校，并且我们也在考虑未来在社区内建立教育设施。

\d+\.  
[客户问题] 我有老人需要照顾，有便利吗？
[销售回答] 我们小区有无障碍通道和专门的老年活动中心，非常适合老年人。

\d+\.  
[客户问题] 这里会不会很吵？
[销售回答] 我们有良好的隔音设计和规划，内部环境非常宁静。



In [26]:
db_real.save_local("real_estate_sales")

In [27]:
db_english.save_local("english_training_sales")

In [28]:
db_iphonoe.save_local("iphone_sales")

In [59]:
db_car.save_local("car_sales")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [29]:
db_real_topK_retriever = db_real.as_retriever(search_kwargs={"k": 3})

In [30]:
db_english_topK_retriever = db_english.as_retriever(search_kwargs={"k": 3})

In [31]:
db_iphonoe_topK_retriever = db_iphonoe.as_retriever(search_kwargs={"k": 3})

In [33]:
db_real_topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x117fa85b0>, search_kwargs={'k': 3})

In [34]:
db_english_topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x117fb98d0>, search_kwargs={'k': 3})

In [35]:
db_iphonoe_topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x117fcefe0>, search_kwargs={'k': 3})

In [36]:
docs = db_real_topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

/var/folders/hs/8p529r5x2811ltb4jwc49lgr0000gn/T/ipykernel_8364/3884148054.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = db_real_topK_retriever.get_relevant_documents(query)


[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

\d+\.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

[客户问题] 我喜欢安静，这里噪音大吗？
[销售回答] 我们特意进行了隔音设计，并且小区内部也有绿化带，整体非常安静。

\d+\.  
[客户问题] 我看这个小区还在建设中。
[销售回答] 是的，目前正在建设中，但已经接近尾声，很快就会完工。

\d+\.  
[客户问题] 有哪些休闲设施？
[销售回答] 我们有游泳池、健身房、图书馆以及儿童游乐场等多种休闲设施。

[客户问题] 我听说这个区域交通常常堵塞。
[销售回答] 我们小区的位置实际上是绕开了主要交通瓶颈区，所以相对来说会流畅很多。

\d+\.  
[客户问题] 我很关注环境保护，你们有做到吗？
[销售回答] 非常好的问题，我们的建筑采用环保材料，并且在小区规划中也有绿化区域和垃圾分类系统。



In [37]:
docs = db_english_topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 是否有校友网络或协会？
[销售回答] 是的，我们有校友网络和协会，这将为您提供与其他校友建立联系和分享职业机会的机会。

\d+\.
[客户问题] 培训课程是否提供贷款或金融支持？
[销售回答] 我们可以为符合条件的学员提供贷款或金融支持选项，以帮助您支付课程费用。请与我们的财务团队联系以获取详细信息。

[客户问题] 是否提供在线讨论板或社交媒体群组？
[销售回答] 是的，我们提供在线讨论板和社交媒体群组，以便学员之间交流和互动。这是一个方便的方式来分享知识和资源。

\d+\.
[客户问题] 培训课程是否提供实习安排？
[销售回答] 是的，一些我们的课程可以帮助学员安排实习机会，让他们在实际工作中应用所学知识。我们将提供支持来寻找合适的实习机会。

[客户问题] 是否提供在线问答和讨论论坛？
[销售回答] 是的，我们提供在线问答和讨论论坛，以便学员与导师和其他学员互动。这是一个有助于学习和分享经验的平台。

\d+\.
[客户问题] 培训课程是否提供导师反馈？
[销售回答] 是的，我们的导师将定期提供反馈，帮助您了解自己的学习进展并提供改进建议。这有助于您不断提高。



In [38]:
docs = db_iphonoe_topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 它支持双卡双待吗？
[销售回答] 是的，它支持双卡双待功能，可以同时使用两张SIM卡，非常适合经常需要切换不同网络的用户。

\d+\.
[客户问题] 电池续航怎么样？
[销售回答] 电池续航非常优秀，可以轻松应对一整天的正常使用。并且配备快充功能，能在短时间内快速充电。

[客户问题] 它有防水功能吗？
[销售回答] 是的，这款iPhone具有卓越的防水和防尘功能，能够抵抗日常生活中的水和尘土。

\d+\.
[客户问题] 它的音质如何？
[销售回答] iPhone具有出色的音质，配备了高品质的扬声器和微型话筒，无论是听音乐还是进行通话，都能提供清晰而饱满的声音体验。

[客户问题] 我可以在iPhone上看电视节目吗？
[销售回答] 当然可以，您可以通过各种流媒体应用，如Netflix, Hulu或Apple TV来观看您喜欢的电视节目和电影。

\d+\.
[客户问题] 它的音质如何？
[销售回答] 它配备了高质量的扬声器和微型麦克风，可以提供出色的音频体验，无论是听音乐还是进行通话。



In [39]:
docs = db_real_topK_retriever.get_relevant_documents("你们有没有1000万的豪宅啊？")

In [40]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

\d+\.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。

\d+\.  
[客户问题] 你们是否提供按揭服务？
[销售回答] 是的，我们与多家银行合作，可以帮助您快速办理按揭。

[客户问题] 这个价位对我来说有点高。
[销售回答] 我们有不同户型和付款方案，一定有适合您预算的。

\d+\.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到业主一致好评，服务周到且高效。

\d+\.  
[客户问题] 如果我想转卖这个房子呢？
[销售回答] 我们的房子位于黄金地段，升值潜力很大，转卖起来不会有问题。

[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

\d+\.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

\d+\.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。



In [41]:
docs = db_iphonoe_topK_retriever.get_relevant_documents("这款手机有抗摔功能吗？")
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 为什么它这么贵？
[销售回答] iPhone采用了最先进的技术和高品质的材料来保证您有最好的使用体验，这也是它价格相对较高的一个重要原因。

\d+\.
[客户问题] 这款手机有抗摔功能吗？
[销售回答] iPhone设计非常坚固，采用了陶瓷防护屏幕，提供了很好的抗摔性能。但还是建议配合保护壳来使用以确保更长的使用寿命。

[客户问题] 它的屏幕是防刮的吗？
[销售回答] 是的，它的屏幕采用特殊的材料制造，具有一定的抗刮擦能力，但我们仍然推荐您使用屏幕保护膜来提供额外的保护。

\d+\.
[客户问题] 它支持5G网络吗？
[销售回答] 是的，它支持5G网络，可以让您在支持的区域享受极速的网络连接。

[客户问题] 它有防水功能吗？
[销售回答] 是的，这款iPhone具有卓越的防水和防尘功能，能够抵抗日常生活中的水和尘土。

\d+\.
[客户问题] 它的音质如何？
[销售回答] iPhone具有出色的音质，配备了高品质的扬声器和微型话筒，无论是听音乐还是进行通话，都能提供清晰而饱满的声音体验。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [51]:
retriever = db_real.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

No relevant docs were retrieved using the relevance score threshold 0.8


In [54]:
retriever = db_iphonoe.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.7}
)

docs = retriever.get_relevant_documents("这款手机有抗摔功能吗？")
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 为什么它这么贵？
[销售回答] iPhone采用了最先进的技术和高品质的材料来保证您有最好的使用体验，这也是它价格相对较高的一个重要原因。

\d+\.
[客户问题] 这款手机有抗摔功能吗？
[销售回答] iPhone设计非常坚固，采用了陶瓷防护屏幕，提供了很好的抗摔性能。但还是建议配合保护壳来使用以确保更长的使用寿命。

[客户问题] 它的屏幕是防刮的吗？
[销售回答] 是的，它的屏幕采用特殊的材料制造，具有一定的抗刮擦能力，但我们仍然推荐您使用屏幕保护膜来提供额外的保护。

\d+\.
[客户问题] 它支持5G网络吗？
[销售回答] 是的，它支持5G网络，可以让您在支持的区域享受极速的网络连接。

[客户问题] 它有防水功能吗？
[销售回答] 是的，这款iPhone具有卓越的防水和防尘功能，能够抵抗日常生活中的水和尘土。

\d+\.
[客户问题] 它的音质如何？
[销售回答] iPhone具有出色的音质，配备了高品质的扬声器和微型话筒，无论是听音乐还是进行通话，都能提供清晰而饱满的声音体验。

[客户问题] 这款手机的相机功能如何？
[销售回答] 该款iPhone配备了先进的相机系统，包含了多个后置摄像头和一个前置摄像头，可以捕捉到生动清晰的照片和视频。其高动态范围和夜间模式等功能更是确保您在任何光线条件下都可以获得卓越的拍摄效果。



## 测试新增Car 种类的查询

In [62]:
retriever = db_car.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.6}
)

docs = retriever.get_relevant_documents("刹车油多久换一次")
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 全合成机油必须半年换一次吗？  
[销售回答] 全合成机油更换周期可达1年或1万公里，但需根据实际驾驶环境和机油衰减情况调整。  

\d+\.  
[客户问题] 车辆年检需要准备哪些材料？  
[销售回答] 需携带行驶证、交强险保单、车主身份证，7座以下新车6年内免上线检测。

[客户问题] 刹车片多久需要更换？  
[销售回答] 通常前刹车片3-6万公里更换，后刹车片6-10万公里，具体需根据磨损提示器判断。  

\d+\.  
[客户问题] 车内异味如何快速去除？  
[销售回答] 建议多通风暴晒，使用活性炭包，部分车型配备空气净化系统可加速异味消散。

[客户问题] 变速箱油需要更换吗？  
[销售回答] 自动变速箱油建议6-8万公里更换，手动变速箱可延长至8-10万公里。  

\d+\.  
[客户问题] 四驱车油耗比两驱高多少？  
[销售回答] 城市路况约高10%-15%，高速路况差异较小，部分智能四驱可自动切换降低能耗。

[客户问题] 车辆灯光调节高度怎么设置？  
[销售回答] 车内配备大灯高度调节滚轮，部分车型自动调节，具体参考说明书操作。  

\d+\.  
[客户问题] 刹车油多久换一次？  
[销售回答] 每2年或4万公里更换，吸水性强易影响制动性能，需定期检测含水量。

